# **Importing Model building libraries**

In [19]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras_preprocessing import sequence
from keras.utils import to_categorical
from keras.models import load_model

# **Importing NLTK libraries**

In [6]:
import csv
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
nltk.download('stopwords')  
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# **Reading dataset and preprocessing**

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
df = pd.read_csv('/content/gdrive/MyDrive/spam.csv',delimiter=',',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [9]:
df.drop(['Unnamed: 2','Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True) 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [10]:
df.groupby(['v1']).size()

v1
ham     4825
spam     747
dtype: int64

In [21]:
#Label Encoding Required Column
X = df.v2
Y = df.v1
le = preprocessing.LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [22]:
# Test and train data split 
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [23]:
# Tokenisation function
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)


# **Create Model**
# **Add layers (LSTM ,Dense-(HiddenLayers),Ouput)**

In [24]:
#LSTM model
inputs = Input(name='InputLayer',shape=[max_len])
layer = Embedding(max_words,50,input_length=max_len)(inputs)
layer = LSTM(64)(layer)
layer = Dense(256,name='FullyConnectedLayer1')(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(1,name='OutputLayer')(layer)
layer = Activation('sigmoid')(layer)

In [25]:
model = Model(inputs=inputs,outputs=layer)
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 InputLayer (InputLayer)     [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FullyConnectedLayer1 (Dense  (None, 256)              16640     
 )                                                               
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                             

In [26]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=25,validation_split=0.2)

Epoch 1/25
30/30 [==============================] - 11s 274ms/step - loss: 0.3376 - accuracy: 0.8659 - val_loss: 0.1359 - val_accuracy: 0.9789
Epoch 2/25
30/30 [==============================] - 9s 317ms/step - loss: 0.0902 - accuracy: 0.9762 - val_loss: 0.0471 - val_accuracy: 0.9863
Epoch 3/25
30/30 [==============================] - 8s 256ms/step - loss: 0.0435 - accuracy: 0.9863 - val_loss: 0.0401 - val_accuracy: 0.9863
Epoch 4/25
30/30 [==============================] - 8s 256ms/step - loss: 0.0322 - accuracy: 0.9886 - val_loss: 0.0431 - val_accuracy: 0.9884
Epoch 5/25
30/30 [==============================] - 8s 259ms/step - loss: 0.0256 - accuracy: 0.9923 - val_loss: 0.0563 - val_accuracy: 0.9863
Epoch 6/25
30/30 [==============================] - 10s 333ms/step - loss: 0.0176 - accuracy: 0.9947 - val_loss: 0.1245 - val_accuracy: 0.9789
Epoch 7/25
30/30 [==============================] - 8s 257ms/step - loss: 0.0180 - accuracy: 0.9952 - val_loss: 0.0629 - val_accuracy: 0.9863
Epoc

In [27]:
model.save("AI_Spam_Identifier")

In [28]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [29]:
accuracy = model.evaluate(test_sequences_matrix,Y_test)
print('Accuracy: {:0.3f}'.format(accuracy[1]))

27/27 [==============================] - 1s 21ms/step - loss: 0.1908 - accuracy: 0.9833
Accuracy: 0.983


In [30]:
y_pred = model.predict(test_sequences_matrix)
print(y_pred[25:40].round(3))

27/27 [==============================] - 1s 22ms/step
[[1.   ]
 [1.   ]
 [0.   ]
 [0.   ]
 [1.   ]
 [1.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [1.   ]
 [0.001]
 [0.   ]
 [0.   ]]


In [31]:
print(Y_test[25:40])

[[1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]]
